# Making grids for exporting and estimating secondary forest and pastureland areas

Intakes:

MapBiomas Land Use
MapBiomas Secondary Forest Age

In [2]:
import ee
import geemap
from utils import *
initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year

## Estimate total area

- Total secondary forest area per 1km2
- Total pasture area per 1km2

In [3]:
lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
            .select([f"classification_{year}" for year in config.range_1985_2020])
            .byte()
            .rename([str(year) for year in config.range_1985_2020]))

pastureland = lulc.select("2020").eq(15).unmask(0).rename("pastureland")

age = ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_secondary_vegetation_age_v1").select("secondary_vegetation_age_2020")

# Aggregate the high-resolution pixels into the 1km grid
def estimate_area(image, name):

    biomes = ee.Image(f"{config.data_folder}/categorical").select("biome")

    image_mask = image.gt(0).And(biomes.eq(1)).unmask(0)

    image_scale = round(image.projection().nominalScale().getInfo())
    closest_multiple = round(1000 / image_scale) * image_scale

    # First, sample locations based only on the age band
    grid = geemap.create_grid(image.geometry(), closest_multiple, image.projection())

    # Create an image representing pixel area
    area_image = ee.Image.pixelArea().updateMask(image_mask)

    # Function to compute valid area per grid cell
    def compute_area(feature):
        valid_area = area_image.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=feature.geometry(),
            crs=image.projection(),
            crsTransform=image.projection().getInfo()['transform'],
            maxPixels=1e10
        ).getNumber('area')
        return feature.set('valid_area', valid_area)

    # Apply to grid FeatureCollection
    grid_with_area = grid.map(compute_area)

    area_image_raster = grid_with_area.reduceToImage(
        properties=['valid_area'],
        reducer=ee.Reducer.first()
    )

    task = ee.batch.Export.image.toAsset(
        image = area_image_raster,
        description = f"{name}_area_1km",
        assetId = f"projects/amazon-forest-regrowth/assets/{name}_area_1km",
        crs = image.projection().crs(),
        crsTransform=image.projection().atScale(990).getInfo()['transform'],
        region = image.geometry()
    )
    task.start()


# estimate_area(pastureland, "pastureland")
# estimate_area(age.gt(0).unmask(0), "secondary")

## Make grid to export one pixel per 10km2 and one pixel per km2
- 1 secondary forest pixel per 1km2
- 1 pasture pixel per 1km2

In [4]:
def create_grid(image, region_name = "amazon", cell_size = 10000, file_name = None):
    
    biomes = ee.Image(f"{config.data_folder}/categorical").select("biome")
    
    if region_name == "amazon":
        biome_index = 1
    elif region_name == "atlantic":
        biome_index = 4
    
    biomes = biomes.eq(biome_index).selfMask()

    # pixels_to_sample = image.gt(0).updateMask(biomes).gt(0).rename("biome")
    pixels_to_sample = biomes.reduceResolution(
        ee.Reducer.first(), maxPixels=65536
        ).reproject(
        crs = image.projection().getInfo()['crs'],
        crsTransform = image.projection().getInfo()['transform']
        ).updateMask(image)
    

    image_scale = round(pixels_to_sample.projection().nominalScale().getInfo())
    closest_multiple = round(cell_size / image_scale) * image_scale

    # First, sample locations based only on the age band
    grid = geemap.create_grid(pixels_to_sample.geometry(), closest_multiple, pixels_to_sample.projection())

    # Function to sample one point per valid cell
    def sample_cell(cell):
        sampled_fc = pixels_to_sample.stratifiedSample(
            numPoints = 1,
            classBand = 'biome',
            region = cell.geometry(),
            scale = pixels_to_sample.projection().nominalScale(),
            geometries = True,
            dropNulls = True,
            tileScale = 3
        )

        # Only return a feature if we found one
        return ee.Feature(ee.Algorithms.If(
            sampled_fc.size().gt(0),
            sampled_fc.first(),
            # Return a placeholder that we can filter out later
            ee.Feature(ee.Geometry.Point([0, 0])).set('is_null', True)
        ))

    samples = grid.map(sample_cell)

    # Filter out placeholder features before exporting
    samples = samples.filter(ee.Filter.notEquals('is_null', True))

    if file_name is None:
        return samples
    else:
        export_name = f"grid_{cell_size//1000}k_{region_name}_{file_name}"

        export_task = ee.batch.Export.table.toAsset(
            collection = samples,
            description = export_name,
            assetId = f"{config.data_folder}/{export_name}"
        )
        export_task.start()



In [5]:
age = ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_secondary_vegetation_age_v1").select("secondary_vegetation_age_2020")

distance_to_border_mask = ee.Image(f"{data_folder}/distance_to_border_mask")

age = age.updateMask(distance_to_border_mask).rename("age")

pastureland = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
            .select([f"classification_{year}" for year in config.range_1985_2020])
            .byte()
            .rename([str(year) for year in config.range_1985_2020]))
pastureland = pastureland.select("2020").eq(15).unmask(0).rename("pastureland")

In [9]:
distance_to_secondary = ee.Image(f"{data_folder}/distance_to_secondary").lt(30)

# create_grid(age.gt(0).unmask(0), region_name = "amazon", cell_size = 10000, file_name = "secondary_allpixels")

# create_grid(age.updateMask(distance_to_secondary), region_name = "amazon", cell_size = 10000, file_name = "secondary_edge_removed")
# create_grid(pastureland, region_name = "amazon", cell_size = 10000, file_name = "pastureland")
# create_grid(pastureland, region_name = "amazon", cell_size = 1000, file_name = "pastureland")

# make one grid with all pixels that are NOT mature forests for nearest mature forest estimates
# mature_biomass_10k = ee.Image(f"{data_folder}/mature_biomass_10k")
# non_mature_forests = mature_biomass_10k.eq(0).unmask(1).selfMask().rename("non_mature_forests")
# create_grid(non_mature_forests, "amazon", cell_size = 10000, file_name = "non_mature_forests")